In [1]:
import os
import shutil
import numpy as np
from collections import defaultdict
from PIL import Image
import pillow_heif
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.models import Model

Num GPUs Available:  0


# Convert HEIC to JPG

In [2]:
import os
import shutil
from PIL import Image
import pillow_heif

def convert_to_jpeg(image_folder):
    print(f"Converting images in folder: {image_folder}")
    
    # Create a folder for videos if it doesn't exist
    videos_folder = os.path.join(image_folder, 'videos')
    os.makedirs(videos_folder, exist_ok=True)

    # List files in the image folder for verification
    print("Files in the folder:", os.listdir(image_folder))

    # Loop through all files in the image folder
    for image_file in os.listdir(image_folder):
        # Get the full path of the image file
        image_path = os.path.join(image_folder, image_file)

        # Process only files
        if os.path.isfile(image_path):
            # Get the file extension
            _, ext = os.path.splitext(image_file)

            # Check if it's a video file
            if ext.lower() in ['.mp4', '.avi', '.mov', '.mkv', '.wmv', '.flv']:
                # Move video file to the videos folder
                shutil.move(image_path, os.path.join(videos_folder, image_file))
                print(f"Moved video file to {videos_folder}: {image_file}")
            # Convert HEIC and other images to JPEG
            elif ext.lower() in ['.heic', '.jpeg', '.jpg', '.png', '.gif']:
                try:
                    # Open HEIC image
                    if ext.lower() == '.heic':
                        heif_file = pillow_heif.open_heif(image_path)
                        img = Image.frombytes('RGB', heif_file.size, heif_file.data)
                    else:
                        img = Image.open(image_path)

                    # Convert image to RGB (JPEG format)
                    img = img.convert('RGB')
                    
                    # Save the image as JPEG in the same folder
                    output_file = os.path.join(image_folder, f"{os.path.splitext(image_file)[0]}.jpg")
                    img.save(output_file, format='JPEG')
                    print(f"Converted {image_file} to {output_file}")

                    # If the conversion is successful, delete the original HEIC file
                    if ext.lower() == '.heic':
                        os.remove(image_path)
                        print(f"Deleted original HEIC file: {image_file}")
                except Exception as e:
                    print(f"Error converting {image_file}: {e}")

# Define paths
image_folder = './images/'  # Folder containing images

# Convert images
convert_to_jpeg(image_folder)


Converting images in folder: ./images/
Files in the folder: ['IMG_0323.MOV', 'IMG_0324.MOV', 'IMG_0326.MOV', 'IMG_0327.MOV', 'IMG_0328.MOV', 'IMG_0329.MOV', 'IMG_0330.MOV', 'IMG_0331.MOV', 'IMG_0332.MOV', 'IMG_0333(1).HEIC', 'IMG_0333.HEIC', 'IMG_0334.HEIC', 'IMG_0335.HEIC', 'IMG_0336.HEIC', 'IMG_0337.HEIC', 'IMG_0338.MOV', 'IMG_0339.HEIC', 'IMG_0341.HEIC', 'IMG_0342.MOV', 'IMG_0343.MOV', 'IMG_0344.MOV', 'IMG_0345.JPG', 'IMG_0346.JPG', 'IMG_0348.MOV', 'IMG_0349.MOV', 'IMG_0350.HEIC', 'IMG_0351.HEIC', 'IMG_0352.HEIC', 'IMG_0353.HEIC', 'IMG_0354.HEIC', 'IMG_0355.HEIC', 'IMG_0356.HEIC', 'IMG_0357.HEIC', 'IMG_0358.HEIC', 'IMG_0359.MOV', 'IMG_0361.HEIC', 'IMG_0362.HEIC', 'IMG_0363.HEIC', 'IMG_0364.HEIC', 'IMG_0365.MOV', 'IMG_0366.HEIC', 'IMG_0367.HEIC', 'IMG_0368.HEIC', 'IMG_0369.HEIC', 'IMG_0370.HEIC', 'IMG_0371.JPG', 'IMG_0372.JPG', 'IMG_0373.JPG', 'IMG_0374.JPG', 'IMG_0375.MOV', 'IMG_0376.JPG', 'IMG_0377.MOV', 'IMG_0378.MOV', 'IMG_0379.MOV', 'IMG_0380.MOV', 'IMG_0381.MOV', 'IMG_0382.HEIC

Converted IMG_0404.HEIC to ./images/IMG_0404.jpg
Deleted original HEIC file: IMG_0404.HEIC
Moved video file to ./images/videos: IMG_0405.MOV
Converted IMG_0406.HEIC to ./images/IMG_0406.jpg
Deleted original HEIC file: IMG_0406.HEIC
Converted IMG_0407.HEIC to ./images/IMG_0407.jpg
Deleted original HEIC file: IMG_0407.HEIC
Converted IMG_0408.HEIC to ./images/IMG_0408.jpg
Deleted original HEIC file: IMG_0408.HEIC
Converted IMG_0409.HEIC to ./images/IMG_0409.jpg
Deleted original HEIC file: IMG_0409.HEIC
Moved video file to ./images/videos: IMG_0410.MOV
Moved video file to ./images/videos: IMG_0411.MOV
Moved video file to ./images/videos: IMG_0412.MOV
Moved video file to ./images/videos: IMG_0413.MOV
Converted IMG_0414.JPG to ./images/IMG_0414.jpg
Moved video file to ./images/videos: IMG_0415.MOV
Converted IMG_0416.HEIC to ./images/IMG_0416.jpg
Deleted original HEIC file: IMG_0416.HEIC
Moved video file to ./images/videos: IMG_0417.MOV
Moved video file to ./images/videos: IMG_0418.MOV
Conver

# Feature Extraction Using VGG16

In [3]:
def extract_features(image_path):
    # Load VGG16 model without the top layer (only the feature extraction part)
    base_model = VGG16(weights='imagenet', include_top=False)
    model = Model(inputs=base_model.input, outputs=base_model.output)

    # Load and preprocess the image
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize to VGG16 input size
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Extract features
    features = model.predict(img_array)
    features = features.flatten()  # Flatten to 1D array
    return features


# Identify Duplicates

In [4]:
def identify_duplicates(image_folder, dupes_folder, best_folder, threshold=0.5):
    image_features = defaultdict(list)
    
    # Extract features for all images
    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_folder, filename)
            features = extract_features(image_path)
            image_features[filename] = features

    # Create the dupes and best folders if they don't exist
    os.makedirs(dupes_folder, exist_ok=True)
    os.makedirs(best_folder, exist_ok=True)

    kept_images = []
    checked = set()

    for img1, features1 in image_features.items():
        if img1 in checked:
            continue
        kept_images.append(img1)
        checked.add(img1)

        for img2, features2 in image_features.items():
            if img1 != img2 and img2 not in checked:
                # Calculate similarity using cosine similarity
                similarity = np.dot(features1, features2) / (np.linalg.norm(features1) * np.linalg.norm(features2))
                print(f"Comparing {img1} and {img2}: Similarity = {similarity:.4f}")  # Debugging output
                
                if similarity > threshold:
                    img2_path = os.path.join(image_folder, img2)
                    if os.path.exists(img2_path):  # Check if the image exists before moving
                        shutil.move(img2_path, os.path.join(dupes_folder, img2))
                        print(f"Moved duplicate image to dupes folder: {img2}")

    # Move kept images to the best folder
    for kept_image in kept_images:
        kept_image_path = os.path.join(image_folder, kept_image)
        if os.path.exists(kept_image_path):  # Check if the image exists before moving
            shutil.move(kept_image_path, os.path.join(best_folder, kept_image))
            print(f"Moved kept image to best folder: {kept_image}")


# Run Duplicate Identification

In [5]:
# Define output folders
dupes_folder = './images/dupes/'  # Folder to save duplicate images
best_folder = './images/best/'      # Folder to save best images

# Identify duplicates
identify_duplicates(image_folder, dupes_folder, best_folder)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Comparing IMG_0333(1).jpg and IMG_0333.jpg: Similarity = 1.0000
Moved duplicate image to dupes folder: IMG_0333.jpg
Comparing IMG_0333(1).jpg and IMG_0334.jpg: Similarity = 0.0724
Comparing IMG_0333(1).jpg and IMG_0335.jpg: Similarity = 0.0792
Comparing IMG_0333(1).jpg and IMG_0336.jpg: Similarity = 0.0454
Comparing IMG_0333(1).jpg and IMG_0337.jpg: Similarity = 0.0561
Comparing IMG_0333(1).jpg and IMG_0339.jpg: Similarity = 0.0315
Comparing IMG_0333(1).jpg and IMG_0341.jpg: Similarity = 0.0353
Comparing IMG_0333(1).jpg and IMG_0345.JPG: Similarity = 0.0613
Comp

Comparing IMG_0354.jpg and IMG_0389.jpg: Similarity = 0.0541
Comparing IMG_0354.jpg and IMG_0390.jpg: Similarity = 0.0700
Comparing IMG_0354.jpg and IMG_0391.jpg: Similarity = 0.0493
Comparing IMG_0354.jpg and IMG_0392.jpg: Similarity = 0.0660
Comparing IMG_0354.jpg and IMG_0393.jpg: Similarity = 0.0632
Comparing IMG_0354.jpg and IMG_0395.jpg: Similarity = 0.0926
Comparing IMG_0354.jpg and IMG_0396.jpg: Similarity = 0.0838
Comparing IMG_0354.jpg and IMG_0397.jpg: Similarity = 0.0821
Comparing IMG_0354.jpg and IMG_0398.jpg: Similarity = 0.0933
Comparing IMG_0354.jpg and IMG_0399.jpg: Similarity = 0.0984
Comparing IMG_0354.jpg and IMG_0401.jpg: Similarity = 0.0815
Comparing IMG_0354.jpg and IMG_0402.jpg: Similarity = 0.0989
Comparing IMG_0354.jpg and IMG_0403.jpg: Similarity = 0.0905
Comparing IMG_0354.jpg and IMG_0404.jpg: Similarity = 0.0909
Comparing IMG_0354.jpg and IMG_0406.jpg: Similarity = 0.0756
Comparing IMG_0354.jpg and IMG_0407.jpg: Similarity = 0.0942
Comparing IMG_0354.jpg a

Comparing IMG_0362.jpg and IMG_0374.JPG: Similarity = 0.1447
Comparing IMG_0362.jpg and IMG_0376.JPG: Similarity = 0.2006
Comparing IMG_0362.jpg and IMG_0382.jpg: Similarity = 0.0717
Comparing IMG_0362.jpg and IMG_0383.jpg: Similarity = 0.0611
Comparing IMG_0362.jpg and IMG_0384.jpg: Similarity = 0.0802
Comparing IMG_0362.jpg and IMG_0385.jpg: Similarity = 0.0796
Comparing IMG_0362.jpg and IMG_0386.jpg: Similarity = 0.0315
Comparing IMG_0362.jpg and IMG_0389.jpg: Similarity = 0.0447
Comparing IMG_0362.jpg and IMG_0390.jpg: Similarity = 0.0733
Comparing IMG_0362.jpg and IMG_0391.jpg: Similarity = 0.0470
Comparing IMG_0362.jpg and IMG_0392.jpg: Similarity = 0.0258
Comparing IMG_0362.jpg and IMG_0393.jpg: Similarity = 0.0312
Comparing IMG_0362.jpg and IMG_0395.jpg: Similarity = 0.0918
Comparing IMG_0362.jpg and IMG_0396.jpg: Similarity = 0.0925
Comparing IMG_0362.jpg and IMG_0397.jpg: Similarity = 0.0813
Comparing IMG_0362.jpg and IMG_0398.jpg: Similarity = 0.1013
Comparing IMG_0362.jpg a

Comparing IMG_0376.JPG and IMG_0469.jpg: Similarity = 0.0635
Comparing IMG_0376.JPG and IMG_0470.jpg: Similarity = 0.0633
Comparing IMG_0376.JPG and IMG_0472.jpg: Similarity = 0.0952
Comparing IMG_0376.JPG and IMG_0473.jpg: Similarity = 0.0865
Comparing IMG_0376.JPG and IMG_0474.jpg: Similarity = 0.1010
Comparing IMG_0376.JPG and IMG_0475.jpg: Similarity = 0.0715
Comparing IMG_0376.JPG and IMG_0476.jpg: Similarity = 0.0615
Comparing IMG_0376.JPG and IMG_0477.jpg: Similarity = 0.0639
Comparing IMG_0376.JPG and IMG_0478.jpg: Similarity = 0.0632
Comparing IMG_0376.JPG and IMG_0480.jpg: Similarity = 0.0620
Comparing IMG_0382.jpg and IMG_0383.jpg: Similarity = 0.3804
Comparing IMG_0382.jpg and IMG_0384.jpg: Similarity = 0.3106
Comparing IMG_0382.jpg and IMG_0385.jpg: Similarity = 0.3208
Comparing IMG_0382.jpg and IMG_0386.jpg: Similarity = 0.2099
Comparing IMG_0382.jpg and IMG_0389.jpg: Similarity = 0.2476
Comparing IMG_0382.jpg and IMG_0390.jpg: Similarity = 0.3408
Comparing IMG_0382.jpg a

Comparing IMG_0395.jpg and IMG_0403.jpg: Similarity = 0.2929
Comparing IMG_0395.jpg and IMG_0404.jpg: Similarity = 0.3108
Comparing IMG_0395.jpg and IMG_0406.jpg: Similarity = 0.1997
Comparing IMG_0395.jpg and IMG_0407.jpg: Similarity = 0.1532
Comparing IMG_0395.jpg and IMG_0408.jpg: Similarity = 0.0284
Comparing IMG_0395.jpg and IMG_0409.jpg: Similarity = 0.0717
Comparing IMG_0395.jpg and IMG_0414.JPG: Similarity = 0.0505
Comparing IMG_0395.jpg and IMG_0416.jpg: Similarity = 0.1362
Comparing IMG_0395.jpg and IMG_0419.jpg: Similarity = 0.0541
Comparing IMG_0395.jpg and IMG_0420.jpg: Similarity = 0.0503
Comparing IMG_0395.jpg and IMG_0421.jpg: Similarity = 0.0531
Comparing IMG_0395.jpg and IMG_0422.jpg: Similarity = 0.0270
Comparing IMG_0395.jpg and IMG_0423.jpg: Similarity = 0.0422
Comparing IMG_0395.jpg and IMG_0424.jpg: Similarity = 0.1109
Comparing IMG_0395.jpg and IMG_0425.jpg: Similarity = 0.0947
Comparing IMG_0395.jpg and IMG_0431.jpg: Similarity = 0.1494
Comparing IMG_0395.jpg a

Moved duplicate image to dupes folder: IMG_0432.jpg
Comparing IMG_0431.jpg and IMG_0433.jpg: Similarity = 0.8208
Moved duplicate image to dupes folder: IMG_0433.jpg
Comparing IMG_0431.jpg and IMG_0434.jpg: Similarity = 0.8637
Moved duplicate image to dupes folder: IMG_0434.jpg
Comparing IMG_0431.jpg and IMG_0435.jpg: Similarity = 0.7846
Moved duplicate image to dupes folder: IMG_0435.jpg
Comparing IMG_0431.jpg and IMG_0436.jpg: Similarity = 0.8025
Moved duplicate image to dupes folder: IMG_0436.jpg
Comparing IMG_0431.jpg and IMG_0437.jpg: Similarity = 0.8026
Moved duplicate image to dupes folder: IMG_0437.jpg
Comparing IMG_0431.jpg and IMG_0438.jpg: Similarity = 0.1823
Comparing IMG_0431.jpg and IMG_0439.jpg: Similarity = 0.0821
Comparing IMG_0431.jpg and IMG_0440.jpg: Similarity = 0.2384
Comparing IMG_0431.jpg and IMG_0441.jpg: Similarity = 0.2583
Comparing IMG_0431.jpg and IMG_0442.jpg: Similarity = 0.1461
Comparing IMG_0431.jpg and IMG_0443.jpg: Similarity = 0.1536
Comparing IMG_043

Comparing IMG_0449.jpg and IMG_0473.jpg: Similarity = 0.0838
Comparing IMG_0449.jpg and IMG_0474.jpg: Similarity = 0.0789
Comparing IMG_0449.jpg and IMG_0475.jpg: Similarity = 0.1103
Comparing IMG_0449.jpg and IMG_0476.jpg: Similarity = 0.1003
Comparing IMG_0449.jpg and IMG_0477.jpg: Similarity = 0.0747
Comparing IMG_0449.jpg and IMG_0478.jpg: Similarity = 0.0707
Comparing IMG_0449.jpg and IMG_0480.jpg: Similarity = 0.0666
Comparing IMG_0450.jpg and IMG_0451.jpg: Similarity = 0.6804
Moved duplicate image to dupes folder: IMG_0451.jpg
Comparing IMG_0450.jpg and IMG_0452.jpg: Similarity = 0.1481
Comparing IMG_0450.jpg and IMG_0453.jpg: Similarity = 0.1447
Comparing IMG_0450.jpg and IMG_0454.jpg: Similarity = 0.1387
Comparing IMG_0450.jpg and IMG_0455.jpg: Similarity = 0.1494
Comparing IMG_0450.jpg and IMG_0456.jpg: Similarity = 0.0662
Comparing IMG_0450.jpg and IMG_0458.jpg: Similarity = 0.0513
Comparing IMG_0450.jpg and IMG_0459.jpg: Similarity = 0.0533
Comparing IMG_0450.jpg and IMG_04

Moved kept image to best folder: IMG_0409.jpg
Moved kept image to best folder: IMG_0414.JPG
Moved kept image to best folder: IMG_0416.jpg
Moved kept image to best folder: IMG_0419.jpg
Moved kept image to best folder: IMG_0420.jpg
Moved kept image to best folder: IMG_0422.jpg
Moved kept image to best folder: IMG_0424.jpg
Moved kept image to best folder: IMG_0431.jpg
Moved kept image to best folder: IMG_0438.jpg
Moved kept image to best folder: IMG_0439.jpg
Moved kept image to best folder: IMG_0442.jpg
Moved kept image to best folder: IMG_0447.jpg
Moved kept image to best folder: IMG_0449.jpg
Moved kept image to best folder: IMG_0450.jpg
Moved kept image to best folder: IMG_0452.jpg
Moved kept image to best folder: IMG_0453.jpg
Moved kept image to best folder: IMG_0456.jpg
Moved kept image to best folder: IMG_0458.jpg
Moved kept image to best folder: IMG_0462.jpg
Moved kept image to best folder: IMG_0463.jpg
Moved kept image to best folder: IMG_0464.jpg
Moved kept image to best folder: I

# Face recognition

In [6]:

import face_recognition
import os
import shutil
from PIL import Image


# Function to Detect and Extract Features from Input Image


In [7]:
def get_person_encoding(image_path):
    """
    Extract face encoding (features) from the input image.
    If multiple faces are found, we'll return the first face encoding.
    """
    try:
        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)

        if len(face_encodings) > 0:
            return face_encodings[0]  # Return the first face's encoding
        else:
            print(f"No faces found in {image_path}")
            return None
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None


 # Function to Identify and Move Matching Images

In [8]:
def move_matching_images(person_image_path, person_name, search_folder, dupes_folder, output_folder):
    """
    Search through the images in the search_folder, match faces with the input image (person_image_path),
    and move matching images to a folder named after the person.
    """
    # Get the person's face encoding
    person_encoding = get_person_encoding(person_image_path)
    if person_encoding is None:
        print(f"Could not get face encoding for {person_name}.")
        return
    
    # Create folder for the person if it doesn't exist
    person_folder = os.path.join(output_folder, person_name)
    os.makedirs(person_folder, exist_ok=True)

    # Search for images in the search_folder (excluding dupes)
    for root, _, files in os.walk(search_folder):
        if dupes_folder in root:
            continue  # Skip the dupes folder
        
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, file)
                
                try:
                    # Load the image and extract face encodings
                    image = face_recognition.load_image_file(image_path)
                    face_encodings = face_recognition.face_encodings(image)

                    # Compare each face encoding with the target person's encoding
                    for face_encoding in face_encodings:
                        match = face_recognition.compare_faces([person_encoding], face_encoding, tolerance=0.6)
                        if match[0]:
                            # Move image if a match is found
                            shutil.move(image_path, os.path.join(person_folder, file))
                            print(f"Moved {file} to {person_folder}")
                            break  # Move on to the next image after finding a match
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")


# Testing

In [9]:
import os
import shutil
import face_recognition

# Get user inputs for the reference images and person's name
reference_images = input("Enter the filenames of the reference images (comma-separated): ").split(',')
person_name = input("Enter the name of the person: ")

# Paths
search_folder = r'C:\Users\sulik\Machine-Learning\CloneCatch\images'  # Folder where images are stored
best_folder = os.path.join(search_folder, 'best')  # Folder for best images
output_folder = os.path.join(r'C:\Users\sulik\Machine-Learning\CloneCatch', person_name)  # Folder created with the person's name

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load face encodings for each reference image
person_encodings = []

for img_filename in reference_images:
    img_filename = img_filename.strip()  # Clean up whitespace
    person_image_path = os.path.join(search_folder, img_filename)
    person_image_path_best = os.path.join(best_folder, img_filename)

    # Check if the image file exists in either the 'images' or 'best' folder
    if os.path.exists(person_image_path):
        print(f"Reference image {img_filename} found in {search_folder}.")
        person_image = face_recognition.load_image_file(person_image_path)
    elif os.path.exists(person_image_path_best):
        print(f"Reference image {img_filename} found in {best_folder}.")
        person_image = face_recognition.load_image_file(person_image_path_best)
    else:
        print(f"Error: Reference image {img_filename} not found in {search_folder} or {best_folder}.")
        continue  # Skip this reference image if not found

    # Get the face encodings
    encodings = face_recognition.face_encodings(person_image)
    
    # Ensure at least one face encoding is found
    if len(encodings) > 0:
        person_encodings.append(encodings[0])
        print(f"Encoding generated for {img_filename}.")
    else:
        print(f"No face found in {img_filename}. Skipping.")

# Function to search and move matching images
def search_and_move_images(folder_path):
    for image_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_file)
        
        # Process only image files
        if os.path.isfile(image_path):
            # Load the current image and get its face encodings
            current_image = face_recognition.load_image_file(image_path)
            current_encodings = face_recognition.face_encodings(current_image)
            
            if len(current_encodings) == 0:
                print(f"No faces found in {image_file}. Skipping.")
                continue  # Skip this image if no faces are found
            
            # Check if any face encoding matches any of the person's encodings
            for current_encoding in current_encodings:
                matches = face_recognition.compare_faces(person_encodings, current_encoding)
                
                if True in matches:
                    # If a match is found, move the image to the output folder
                    output_image_path = os.path.join(output_folder, image_file)
                    shutil.copy(image_path, output_image_path)
                    print(f"Copied {image_file} to {output_folder}.")
                    break  # Move to the next image after finding a match

# Search for matching images in both the main search folder and the 'best' folder
print(f"Searching for matching images in {search_folder}...")
search_and_move_images(search_folder)

# Check if the 'best' folder exists and search it as well
if os.path.exists(best_folder):
    print(f"Searching for matching images in {best_folder}...")
    search_and_move_images(best_folder)

print("Search complete.")


Enter the filenames of the reference images (comma-separated): IMG_0438.jpg
Enter the name of the person: Prateesh
Reference image IMG_0438.jpg found in C:\Users\sulik\Machine-Learning\CloneCatch\images\best.
Encoding generated for IMG_0438.jpg.
Searching for matching images in C:\Users\sulik\Machine-Learning\CloneCatch\images...
Searching for matching images in C:\Users\sulik\Machine-Learning\CloneCatch\images\best...
No faces found in IMG_0334.jpg. Skipping.
No faces found in IMG_0339.jpg. Skipping.
No faces found in IMG_0361.jpg. Skipping.
Copied IMG_0376.JPG to C:\Users\sulik\Machine-Learning\CloneCatch\Prateesh.
No faces found in IMG_0389.jpg. Skipping.
No faces found in IMG_0392.jpg. Skipping.
Copied IMG_0395.jpg to C:\Users\sulik\Machine-Learning\CloneCatch\Prateesh.
No faces found in IMG_0406.jpg. Skipping.
No faces found in IMG_0409.jpg. Skipping.
Copied IMG_0414.JPG to C:\Users\sulik\Machine-Learning\CloneCatch\Prateesh.
Copied IMG_0422.jpg to C:\Users\sulik\Machine-Learning\